<a href="https://colab.research.google.com/github/enumura1/jp-text-similarity/blob/main/notebooks/sklearn-catchup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mecab-python3
!pip install unidic-lite

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import MeCab
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# MeCabの初期化
mecab = MeCab.Tagger()

def tokenize_japanese(text):
    """日本語テキストを形態素解析して単語のリストを返す"""
    node = mecab.parseToNode(text)
    words = []
    while node:
        # 品詞情報を取得
        pos = node.feature.split(',')[0]
        # 名詞、動詞、形容詞のみを対象とする
        if pos in ['名詞', '動詞', '形容詞']:
            words.append(node.surface)
        node = node.next
    return ' '.join(words)

# サンプルの日本語テキストデータ
reference_texts = [
    "商品の返品について説明します。返送料は当社負担です。",
    "商品の交換方法についてご案内いたします。",
    "支払い方法は、クレジットカード、銀行振込が利用できます。",
    "配送状況の確認方法をご説明します。",
]

# テキストの前処理
processed_texts = [tokenize_japanese(text) for text in reference_texts]

# ユーザーからの質問例
query = "返品の方法を教えてください"
processed_query = tokenize_japanese(query)

# TF-IDFベクトル化器の初期化
vectorizer = TfidfVectorizer(
    # トークナイズは既に行っているので不要
    tokenizer=lambda x: x.split(),
    analyzer='word'
)

# リファレンステキストをベクトル化
reference_vectors = vectorizer.fit_transform(processed_texts)

# クエリをベクトル化
query_vector = vectorizer.transform([processed_query])

# コサイン類似度の計算
similarities = cosine_similarity(query_vector, reference_vectors).flatten()

# 結果の表示
print("類似度スコア:")
for text, score in zip(reference_texts, similarities):
    print(f"スコア: {score:.4f} - テキスト: {text}")

print("\n最も類似度の高いテキスト:")
best_match_index = similarities.argmax()
print(f"スコア: {similarities[best_match_index]:.4f}")
print(f"テキスト: {reference_texts[best_match_index]}")

# 単語の重要度を確認
feature_names = vectorizer.get_feature_names_out()
print("\n各単語のTF-IDFスコア（クエリ）:")
query_tfidf = query_vector.toarray()[0]
for word, score in zip(feature_names, query_tfidf):
    if score > 0:
        print(f"{word}: {score:.4f}")

# 参考：形態素解析結果の確認
print("\n形態素解析結果:")
print(f"クエリ: {processed_query}")
print("リファレンステキスト:")
for text in processed_texts:
    print(text)

類似度スコア:
スコア: 0.3310 - テキスト: 商品の返品について説明します。返送料は当社負担です。
スコア: 0.1592 - テキスト: 商品の交換方法についてご案内いたします。
スコア: 0.1262 - テキスト: 支払い方法は、クレジットカード、銀行振込が利用できます。
スコア: 0.1592 - テキスト: 配送状況の確認方法をご説明します。

最も類似度の高いテキスト:
スコア: 0.3310
テキスト: 商品の返品について説明します。返送料は当社負担です。

各単語のTF-IDFスコア（クエリ）:
方法: 0.5380
返品: 0.8429

形態素解析結果:
クエリ: 返品 方法 教え ください
リファレンステキスト:
商品 返品 つい 説明 し 返送 当社 負担
商品 交換 方法 つい 案内 いたし
支払い 方法 クレジット カード 銀行 振込 利用 でき
配送 状況 確認 方法 説明 し
